In [ ]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
import os
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data_source = 'kaggle' # alphavantage or kaggle

if data_source == 'alphavantage':
    # ====================== Loading Data from Alpha Vantage ==================================

    api_key = '104SAFLO5FU4FCKF'

    # American Airlines stock market prices
    ticker = "AAL"

    # JSON file with all the stock market data for AAL from the last 20 years
    url_string = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=%s&outputsize=full&apikey=%s"%(ticker,api_key)

    # Save data to this file
    file_to_save = 'stock_market_data-%s.csv'%ticker

    # If you haven't already saved data,
    # Go ahead and grab the data from the url
    # And store date, low, high, volume, close, open values to a Pandas DataFrame
    if not os.path.exists(file_to_save):
        with urllib.request.urlopen(url_string) as url:
            data = json.loads(url.read().decode())
            # extract stock market data
            data = data['Time Series (Daily)']
            df = pd.DataFrame(columns=['Date','Low','High','Close','Open'])
            for k,v in data.items():
                date = dt.datetime.strptime(k, '%Y-%m-%d')
                data_row = [date.date(),float(v['3. low']),float(v['2. high']),
                            float(v['4. close']),float(v['1. open'])]
                df.loc[-1,:] = data_row
                df.index = df.index + 1
        print('Data saved to : %s'%file_to_save)
        df.to_csv(file_to_save)

    # If the data is already there, just load it from the CSV
    else:
        print('File already exists. Loading data from CSV')
        df = pd.read_csv(file_to_save)

else:

    # ====================== Loading Data from Kaggle ==================================
    # You will be using HP's data. Feel free to experiment with other data.
    # But while doing so, be careful to have a large enough dataset and also pay attention to the data normalization
    df = pd.read_csv(os.path.join('/content/drive/MyDrive/Stocks','hpq.us.txt'),delimiter=',',usecols=['Date','Open','High','Low','Close'])
    print('Loaded data from the Kaggle repository')


In [ ]:
# Sort DataFrame by date
df = df.sort_values('Date')

# Double check the result
df.head()


In [ ]:
plt.figure(figsize = (18,9))
plt.plot(range(df.shape[0]),(df['Low']+df['High'])/2.0)
plt.xticks(range(0,df.shape[0],500),df['Date'].loc[::500],rotation=45)
plt.xlabel('Date',fontsize=18)
plt.ylabel('Mid Price',fontsize=18)
plt.show()


In [ ]:
# First calculate the mid prices from the highest and lowest
high_prices = df.loc[:, 'High'].to_numpy()
low_prices = df.loc[:, 'Low'].to_numpy()
mid_prices = (high_prices + low_prices) / 2.0



In [ ]:
train_data = mid_prices[:11000]
test_data = mid_prices[11000:]


In [ ]:
# Scale the data to be between 0 and 1
# When scaling remember! You normalize both test and train data with respect to training data
# Because you are not supposed to have access to test data
scaler = MinMaxScaler()
train_data = train_data.reshape(-1,1)
test_data = test_data.reshape(-1,1)


In [ ]:
# Train the Scaler with training data and smooth data
smoothing_window_size = 2500
for di in range(0,10000,smoothing_window_size):
    scaler.fit(train_data[di:di+smoothing_window_size,:])
    train_data[di:di+smoothing_window_size,:] = scaler.transform(train_data[di:di+smoothing_window_size,:])

# You normalize the last bit of remaining data
scaler.fit(train_data[di+smoothing_window_size:,:])
train_data[di+smoothing_window_size:,:] = scaler.transform(train_data[di+smoothing_window_size:,:])


In [ ]:
# Reshape both train and test data
train_data = train_data.reshape(-1)

# Normalize test data
test_data = scaler.transform(test_data).reshape(-1)


In [ ]:
# Now perform exponential moving average smoothing
# So the data will have a smoother curve than the original ragged data
EMA = 0.0
gamma = 0.1
for ti in range(11000):
  EMA = gamma*train_data[ti] + (1-gamma)*EMA
  train_data[ti] = EMA

# Used for visualization and test purposes
all_mid_data = np.concatenate([train_data,test_data],axis=0)


In [ ]:
window_size = 100
N = train_data.size
std_avg_predictions = []
std_avg_x = []
mse_errors = []

for pred_idx in range(window_size,N):

    if pred_idx >= N:
        date = dt.datetime.strptime(k, '%Y-%m-%d').date() + dt.timedelta(days=1)
    else:
        date = df.loc[pred_idx,'Date']

    std_avg_predictions.append(np.mean(train_data[pred_idx-window_size:pred_idx]))
    mse_errors.append((std_avg_predictions[-1]-train_data[pred_idx])**2)
    std_avg_x.append(date)

print('MSE error for standard averaging: %.5f'%(0.5*np.mean(mse_errors)))


In [ ]:
plt.figure(figsize = (18,9))
plt.plot(range(df.shape[0]),all_mid_data,color='b',label='True')
plt.plot(range(window_size,N),std_avg_predictions,color='orange',label='Prediction')
#plt.xticks(range(0,df.shape[0],50),df['Date'].loc[::50],rotation=45)
plt.xlabel('Date')
plt.ylabel('Mid Price')
plt.legend(fontsize=18)
plt.show()


In [ ]:
window_size = 100
N = train_data.size

run_avg_predictions = []
run_avg_x = []

mse_errors = []

running_mean = 0.0
run_avg_predictions.append(running_mean)

decay = 0.5

for pred_idx in range(1,N):

    running_mean = running_mean*decay + (1.0-decay)*train_data[pred_idx-1]
    run_avg_predictions.append(running_mean)
    mse_errors.append((run_avg_predictions[-1]-train_data[pred_idx])**2)
    run_avg_x.append(date)

print('MSE error for EMA averaging: %.5f'%(0.5*np.mean(mse_errors)))


In [ ]:

plt.figure(figsize = (18,9))
plt.plot(range(df.shape[0]),all_mid_data,color='b',label='True')
plt.plot(range(0,N),run_avg_predictions,color='orange', label='Prediction')
#plt.xticks(range(0,df.shape[0],50),df['Date'].loc[::50],rotation=45)
plt.xlabel('Date')
plt.ylabel('Mid Price')
plt.legend(fontsize=18)
plt.show()


In [ ]:

class DataGeneratorSeq(object):

    def __init__(self,prices,batch_size,num_unroll):
        self._prices = prices
        self._prices_length = len(self._prices) - num_unroll
        self._batch_size = batch_size
        self._num_unroll = num_unroll
        self._segments = self._prices_length //self._batch_size
        self._cursor = [offset * self._segments for offset in range(self._batch_size)]

    def next_batch(self):

        batch_data = np.zeros((self._batch_size),dtype=np.float32)
        batch_labels = np.zeros((self._batch_size),dtype=np.float32)

        for b in range(self._batch_size):
            if self._cursor[b]+1>=self._prices_length:
                #self._cursor[b] = b * self._segments
                self._cursor[b] = np.random.randint(0,(b+1)*self._segments)

            batch_data[b] = self._prices[self._cursor[b]]
            batch_labels[b]= self._prices[self._cursor[b]+np.random.randint(0,5)]

            self._cursor[b] = (self._cursor[b]+1)%self._prices_length

        return batch_data,batch_labels

    def unroll_batches(self):

        unroll_data,unroll_labels = [],[]
        init_data, init_label = None,None
        for ui in range(self._num_unroll):

            data, labels = self.next_batch()

            unroll_data.append(data)
            unroll_labels.append(labels)

        return unroll_data, unroll_labels

    def reset_indices(self):
        for b in range(self._batch_size):
            self._cursor[b] = np.random.randint(0,min((b+1)*self._segments,self._prices_length-1))



dg = DataGeneratorSeq(train_data,5,5)
u_data, u_labels = dg.unroll_batches()

for ui,(dat,lbl) in enumerate(zip(u_data,u_labels)):
    print('\n\nUnrolled index %d'%ui)
    dat_ind = dat
    lbl_ind = lbl
    print('\tInputs: ',dat )
    print('\n\tOutput:',lbl)


In [ ]:
D = 1 # Dimensionality of the data. Since your data is 1-D this would be 1
num_unrollings = 50 # Number of time steps you look into the future.
batch_size = 500 # Number of samples in a batch
num_nodes = [200,200,150] # Number of hidden nodes in each layer of the deep LSTM stack we're using
n_layers = len(num_nodes) # number of layers
dropout = 0.2 # dropout amount

tf.keras.backend.clear_session() # This is important in case you run this multiple times


In [ ]:
train_inputs, train_outputs = [], []

# Simulate input shapes (placeholders are gone in TF 2.x)
# You can append tensors or numpy arrays here
for ui in range(num_unrollings):
    # Use dummy tensors or numpy arrays if needed
    dummy_input = tf.zeros([batch_size, D])  # shape like your original placeholder
    dummy_output = tf.zeros([batch_size, 1]) # shape like your original placeholder
    train_inputs.append(dummy_input)
    train_outputs.append(dummy_output)

In [ ]:
# Define the LSTM layers with dropout support
lstm_layers = [
    tf.keras.layers.LSTM(units=num_nodes[li],
                         kernel_initializer=tf.keras.initializers.GlorotUniform(),
                         return_sequences=True if li < n_layers - 1 else False,  # For multi-layer RNN
                         dropout=dropout,  # Dropout for the input
                         recurrent_dropout=dropout)  # Dropout for the recurrent state
    for li in range(n_layers)
]

# Stack the LSTM layers into a Sequential model or use a Functional API model
model = tf.keras.Sequential(lstm_layers)

# Define the output layer (fully connected layer)
w = tf.Variable(tf.keras.initializers.GlorotUniform()(shape=[num_nodes[-1], 1]))
b = tf.Variable(tf.random.uniform([1], -0.1, 0.1))

In [ ]:
# Define the LSTM cells for the multi-layer RNN
drop_multi_cell = tf.keras.layers.StackedRNNCells([tf.keras.layers.LSTMCell(num_nodes[i]) for i in range(n_layers)])

# Create cell state and hidden state variables to maintain the state of the LSTM
initial_state = []
for li in range(n_layers):
    h = tf.zeros([batch_size, num_nodes[li]])
    c = tf.zeros([batch_size, num_nodes[li]])
    initial_state.append([c, h])  # tf.keras.layers.LSTMCell expects [c, h] tuple

# Prepare inputs in [batch_size, time_steps, input_dim] format
all_inputs = tf.transpose(tf.stack(train_inputs, axis=0), perm=[1, 0, 2])

# Build multi-layer RNN (no time_major here)
rnn_layer = tf.keras.layers.RNN(drop_multi_cell, return_sequences=True, return_state=True)

# Run the RNN
all_lstm_outputs, *state = rnn_layer(all_inputs, initial_state=initial_state)

# all_lstm_outputs shape: [batch_size, time_steps, num_nodes[-1]]
# Reshape to match [batch_size * num_unrollings, num_nodes[-1]]
all_lstm_outputs = tf.reshape(all_lstm_outputs, [batch_size * num_unrollings, num_nodes[-1]])

# Linear output layer
all_outputs = tf.matmul(all_lstm_outputs, w) + b

# Split into [batch_size, output_dim] tensors
split_outputs = tf.split(all_outputs, num_unrollings, axis=0)


In [ ]:
print('Defining training Loss')

# Calculate loss as mean squared error across all unrolling steps
loss = 0.0
for ui in range(num_unrollings):
    loss += tf.reduce_mean(0.5 * (split_outputs[ui] - train_outputs[ui]) ** 2)

print('Learning rate decay operations')

# Learning rate scheduling
global_step = tf.Variable(0, trainable=False, dtype=tf.int64)
initial_learning_rate = 0.01  # example value, replace as needed
min_learning_rate = 0.0001    # example value

# Define decaying learning rate with minimum clip
learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=1,
    decay_rate=0.5,
    staircase=True
)
learning_rate = lambda: tf.maximum(learning_rate(global_step), min_learning_rate)

# Optimizer
print('TF Optimization operations')
optimizer = tf.keras.optimizers.Adam()

# Training step function
@tf.function
def train_step():
    with tf.GradientTape() as tape:
        # Recompute loss inside the tape
        loss_value = 0.0
        for ui in range(num_unrollings):
            loss_value += tf.reduce_mean(0.5 * (split_outputs[ui] - train_outputs[ui]) ** 2)

    gradients = tape.gradient(loss_value, model.trainable_variables)
    gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    global_step.assign_add(1)
    return loss_value

print('\tAll done')


In [ ]:
print('Defining prediction related TF functions')

# Input placeholder is replaced by a tf.function argument or tensor
sample_inputs = tf.keras.Input(shape=(D,), batch_size=1)

# Maintaining LSTM state for prediction stage
sample_c, sample_h, initial_sample_state = [], [], []
for li in range(n_layers):
    sample_c.append(tf.Variable(tf.zeros([1, num_nodes[li]]), trainable=False))
    sample_h.append(tf.Variable(tf.zeros([1, num_nodes[li]]), trainable=False))
    initial_sample_state.append([sample_c[li], sample_h[li]])

# LSTM cell
cells = [tf.keras.layers.LSTMCell(num_nodes[li]) for li in range(n_layers)]
multi_cell = tf.keras.layers.StackedRNNCells(cells)
rnn_layer = tf.keras.layers.RNN(multi_cell, return_state=True, return_sequences=True)

# Function to reset LSTM state
@tf.function
def reset_sample_states():
    for li in range(n_layers):
        sample_c[li].assign(tf.zeros_like(sample_c[li]))
        sample_h[li].assign(tf.zeros_like(sample_h[li]))

# Prediction function
@tf.function
def predict(sample_input):
    input_expanded = tf.expand_dims(tf.expand_dims(sample_input, axis=0), axis=0)  # [1, 1, D]
    initial_state = [(sample_c[li], sample_h[li]) for li in range(n_layers)]
    output, *new_states = rnn_layer(input_expanded, initial_state=initial_state)

    # Update states
    for li in range(n_layers):
        sample_c[li].assign(new_states[li][0])
        sample_h[li].assign(new_states[li][1])

    # Fully connected layer to get prediction
    output_reshaped = tf.reshape(output, [1, -1])  # [1, num_nodes[-1]]
    sample_prediction = tf.matmul(output_reshaped, w) + b
    return sample_prediction

print('\tAll done')


In [ ]:

epochs = 30
valid_summary = 1  # Interval for test predictions
n_predict_once = 50

train_seq_length = train_data.size
train_mse_ot = []
test_mse_ot = []
predictions_over_time = []

# Optimizer and learning rate scheduler
initial_learning_rate = 0.0001
min_learning_rate = 0.000001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.5,
    staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Loss function
mse_loss = tf.keras.losses.MeanSquaredError()

# Data generator
data_gen = DataGeneratorSeq(train_data, batch_size, num_unrollings)
test_points_seq = np.arange(11000, 12000, 50).tolist()
x_axis_seq = []

print("Initialized")

loss_nondecrease_count = 0
loss_nondecrease_threshold = 2

for ep in range(epochs):
    print(f"\nEpoch {ep+1}/{epochs}")
    average_loss = 0

    # === Training ===
    for step in range(train_seq_length // batch_size):
        u_data, u_labels = data_gen.unroll_batches()

        # Stack inputs and labels
        batch_inputs = np.stack(u_data, axis=1).reshape(batch_size, num_unrollings, 1)
        batch_labels = np.stack(u_labels, axis=1).reshape(batch_size, num_unrollings, 1)

        with tf.GradientTape() as tape:
            predictions = model(batch_inputs, training=True)
            loss = mse_loss(batch_labels[:, -1], predictions)  # match target time step

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        average_loss += loss.numpy()

    # === Validation ===
    if (ep + 1) % valid_summary == 0:
        average_loss /= (train_seq_length // batch_size)
        print(f"Average training loss: {average_loss:.6f}")
        train_mse_ot.append(average_loss)

        predictions_seq = []
        mse_test_loss_seq = []

        for w_i in test_points_seq:
            mse_test_loss = 0.0
            our_predictions = []
            x_axis = []

            # Priming the model with past values
            history_input = all_mid_data[w_i - num_unrollings + 1: w_i].reshape(1, num_unrollings - 1, 1)
            for _ in range(num_unrollings - 1):
                _ = model(history_input, training=False)

            # First input to predict
            current_price = all_mid_data[w_i - 1].reshape(1, 1, 1)

            for pred_i in range(n_predict_once):
                pred = model(current_price, training=False)
                pred_val = pred.numpy().flatten()[0]
                our_predictions.append(pred_val)
                current_price = np.array(pred_val).reshape(1, 1, 1)

                if (ep + 1) - valid_summary == 0:
                    x_axis.append(w_i + pred_i)

                true_val = all_mid_data[w_i + pred_i]
                mse_test_loss += 0.5 * (pred_val - true_val) ** 2

            predictions_seq.append(our_predictions)
            mse_test_loss /= n_predict_once
            mse_test_loss_seq.append(mse_test_loss)

            if (ep + 1) - valid_summary == 0:
                x_axis_seq.append(x_axis)

        current_test_mse = np.mean(mse_test_loss_seq)

        if len(test_mse_ot) > 0 and current_test_mse > min(test_mse_ot):
            loss_nondecrease_count += 1
        else:
            loss_nondecrease_count = 0

        if loss_nondecrease_count > loss_nondecrease_threshold:
            lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                optimizer.learning_rate.numpy() * 0.5,
                decay_steps=1000,
                decay_rate=0.5,
                staircase=True
            )
            optimizer.learning_rate = lr_schedule
            loss_nondecrease_count = 0
            print("\tDecreasing learning rate by 0.5")

        test_mse_ot.append(current_test_mse)
        print(f"\tTest MSE: {current_test_mse:.5f}")
        predictions_over_time.append(predictions_seq)
        print("\tFinished Predictions")


In [ ]:
best_prediction_epoch =  28 # replace this with the epoch that you got the best results when running the plotting code

plt.figure(figsize = (18,18))
plt.subplot(2,1,1)
plt.plot(range(df.shape[0]),all_mid_data,color='b')

# Plotting how the predictions change over time
# Plot older predictions with low alpha and newer predictions with high alpha
start_alpha = 0.25
alpha  = np.arange(start_alpha,1.1,(1.0-start_alpha)/len(predictions_over_time[::3]))
for p_i,p in enumerate(predictions_over_time[::3]):
    for xval,yval in zip(x_axis_seq,p):
        plt.plot(xval,yval,color='r',alpha=alpha[p_i])

plt.title('Evolution of Test Predictions Over Time',fontsize=18)
plt.xlabel('Date',fontsize=18)
plt.ylabel('Mid Price',fontsize=18)
plt.xlim(11000,12500)

plt.subplot(2,1,2)

# Predicting the best test prediction you got
plt.plot(range(df.shape[0]),all_mid_data,color='b')
for xval,yval in zip(x_axis_seq,predictions_over_time[best_prediction_epoch]):
    plt.plot(xval,yval,color='r')

plt.title('Best Test Predictions Over Time',fontsize=18)
plt.xlabel('Date',fontsize=18)
plt.ylabel('Mid Price',fontsize=18)
plt.xlim(11000,12500)
plt.show()